In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import tensorflow.keras
import os
checkpoint_path = "training_DenseNet_50/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


In [2]:
df_data=pd.read_csv("DataSetNormalised.csv")

In [3]:
df_label=pd.read_csv("LabelSet.csv")

In [4]:
x=np.array(df_data.iloc[:,1:])

In [5]:
y=np.array(df_label.iloc[:,1])

In [6]:
x=x.astype(np.float32)

In [7]:
x_train=x[:][:10000000]
y_train=y[:][:10000000]
x_test=x[:][10000000:]
y_test=y[:][10000000:]

In [8]:
y_train.shape

(10000000,)

In [11]:
def identity_block(X,no_nodes,Activation):#A custom Residual identity block with a skip connection
    X_shortcut=X
    #There will be 3 layers in total for the sequential connection
    #Activation will be specified by the user as per the Keras Documentation
    #no_nodes denote the size of the model which will be built
    #Applying Batch Normalisation to size the inputs
    #Layer 1 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    #--------------------------------------------------------
    #Layer 2-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    #--------------------------------------------------------
    #Layer 3-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    #--------------------------------------------------------
    #Layer 4-------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X])
    X=tf.keras.layers.Activation('swish')(X)
    #--------------------------------------------------------
    X_shortcut2=X
     #Layer 5 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    #--------------------------------------------------------
    #Layer 6-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    #--------------------------------------------------------
    #Layer 7-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    #--------------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X,X_shortcut2])
    #Layer 8-------------------------------------------------
    X=tf.keras.layers.Activation('swish')(X)
    return X
    

In [14]:
inputs=tf.keras.layers.Input(shape=(28,))                    #Keras input layer
X=tf.keras.layers.Dense(300,activation='swish')(inputs)
X=identity_block(X=X,no_nodes=300,Activation='swish')
X=identity_block(X=X,no_nodes=300,Activation='swish')
X=identity_block(X=X,no_nodes=300,Activation='swish')
X=identity_block(X=X,no_nodes=300,Activation='swish')
X=identity_block(X=X,no_nodes=300,Activation='swish')        
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(X)     #keras output layer
model=tf.keras.Model(inputs,outputs)                         #Model declaration

In [13]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 300)          8700        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 300)          90300       dense[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 300)          1200        dense_1[0][0]                    
_______________________________________________________________________________________

In [15]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.01,
        beta_1=0.9,
        beta_2=0.999),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC']
    )

In [16]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)


In [17]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=10240,
    validation_data=(x_test,y_test),
    callbacks=[cp_callback]
)

Epoch 1/100
456/977 [=============>................] - ETA: 2:06 - loss: 0.6170 - accuracy: 0.6531 - auc: 0.7119

KeyboardInterrupt: 

In [29]:
model.save('ResNet_52')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ResNet_19\assets
